# Average Red Meat vs Fish Intake by Birth Cohort (EU27)

This notebook analyzes generational trends in European dietary intake focusing on red meat and fish. Data source: `data/processed/gdd_cleaned_panel.csv`.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
DATA_FILE = Path('data/processed/gdd_cleaned_panel.csv')
FIG_PATH = Path('figures/eu_avg_redmeat_vs_fish.png')
EU27_ISO3 = [
    'AUT','BEL','BGR','HRV','CYP','CZE','DNK','EST','FIN','FRA','DEU','GRC','HUN','IRL','ITA',
    'LVA','LTU','LUX','MLT','NLD','POL','PRT','ROU','SVK','SVN','ESP','SWE'
]


In [ ]:
def midpoint(age):
    return (np.floor(age / 5) * 5) + 2.5

def derive_cohort(df):
    df = df.copy()
    df['age_mid'] = df['age'].apply(midpoint)
    df['cohort'] = df['year'] - df['age_mid']
    return df


In [ ]:
df = pd.read_csv(DATA_FILE)


In [ ]:
df = derive_cohort(df)
mask = df['food'].isin(['Red meat','Fish']) & df['iso3'].isin(EU27_ISO3)
df_eu = df[mask]


In [ ]:
cohort_avg = (
    df_eu.groupby(['cohort','food'])['median']
    .mean()
    .reset_index()
)
cohort_avg.head()


In [ ]:
df_eu['generation'] = pd.cut(
    df_eu['cohort'],
    bins=[1965, 1980, 1996],
    labels=['Gen X','Millennials'],
    right=True,
    include_lowest=True
)
summary = (
    df_eu.dropna(subset=['generation'])
        .groupby(['generation','food','age_mid'])['median']
        .mean()
        .reset_index()
)
summary.head()


In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
for food, color in [('Red meat','red'),('Fish','green')]:
    for gen, style in [('Millennials','-'),('Gen X','--')]:
        data = summary[(summary['food']==food) & (summary['generation']==gen)]
        data_sorted = data.sort_values('age_mid')
        ax.plot(data_sorted['age_mid'], data_sorted['median'],
                label=f"{food} ({gen})",
                color=color,
                linestyle=style)
ax.set_xlabel('Age (midpoint)')
ax.set_ylabel('Daily Energy Intake (kcal/day)')
ax.set_title('Observed Generational Trend in Red Meat vs Fish Intake (EU27)')
ax.legend()
fig.tight_layout()
FIG_PATH.parent.mkdir(parents=True, exist_ok=True)
fig.savefig(FIG_PATH, dpi=300)
plt.close(fig)
